# Setup

In [1]:
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [4]:
%%time
train_df = pd.read_csv('../../data/train.csv')
test_df = pd.read_csv('../../data/test.csv')

CPU times: user 11.6 s, sys: 1.07 s, total: 12.7 s
Wall time: 13.1 s


In [5]:
var_cols = [f'var_{i}' for i in range(200)]

In [6]:
def unique_counts(df):
    unique_samples = []
    unique_count = np.zeros_like(df)
    
    for feature in tqdm(range(df.shape[1])):
        _, index_, count_ = np.unique(df[:, feature], return_index=True, return_counts=True)
        unique_count[index_[count_ == 1], feature] += 1
    
    return unique_count.sum(axis=1)

In [7]:
test_unique_count = unique_counts(test_df[var_cols].values)
test_real_idx = np.where(test_unique_count != 0)[0]
test_sync_idx = np.where(test_unique_count == 0)[0]

100%|██████████| 200/200 [00:05<00:00, 38.49it/s]


In [8]:
test_df_real = test_df.loc[test_real_idx, :].copy()
test_df_real.reset_index(drop=True, inplace=True)

In [9]:
test_df_sync = test_df.loc[test_sync_idx, :].copy()
test_df_sync.reset_index(drop=True, inplace=True)

In [10]:
train_df['test'] = 0
test_df_real['test'] = 1

In [11]:
full = pd.concat([train_df, test_df_real], axis=0)

# Feature Engineering

## counting

In [12]:
count_var_cols = [f'count_{col}' for col in var_cols]

In [13]:
for col in tqdm(var_cols):
    gdf = full.groupby(col)[col].count()
    full[f'count_{col}'] = full[col].map(gdf)

100%|██████████| 200/200 [01:30<00:00,  1.43it/s]


In [14]:
full.head()

,ID_code,target,test,var_0,var_1,var_10,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109,var_11,var_110,var_111,var_112,var_113,var_114,var_115,var_116,var_117,var_118,var_119,var_12,var_120,var_121,var_122,var_123,var_124,var_125,var_126,var_127,var_128,var_129,var_13,var_130,var_131,var_132,var_133,var_134,var_135,var_136,var_137,var_138,var_139,var_14,var_140,var_141,var_142,var_143,var_144,var_145,var_146,var_147,var_148,var_149,var_15,var_150,var_151,var_152,var_153,var_154,var_155,var_156,var_157,var_158,var_159,var_16,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_17,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_18,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_19,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,var_2,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_3,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_4,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_5,var_50,var_51,var_52,var_53,var_54,var_55,var_56,var_57,var_58,var_59,var_6,var_60,var_61,var_62,var_63,var_64,var_65,var_66,var_67,var_68,var_69,var_7,var_70,var_71,var_72,var_73,var_74,var_75,var_76,var_77,var_78,var_79,var_8,var_80,var_81,var_82,var_83,var_84,var_85,var_86,var_87,var_88,var_89,var_9,var_90,var_91,var_92,var_93,var_94,var_95,var_96,var_97,var_98,var_99,count_var_0,count_var_1,count_var_2,count_var_3,count_var_4,count_var_5,count_var_6,count_var_7,count_var_8,count_var_9,count_var_10,count_var_11,count_var_12,count_var_13,count_var_14,count_var_15,count_var_16,count_var_17,count_var_18,count_var_19,count_var_20,count_var_21,count_var_22,count_var_23,count_var_24,count_var_25,count_var_26,count_var_27,count_var_28,count_var_29,count_var_30,count_var_31,count_var_32,count_var_33,count_var_34,count_var_35,count_var_36,count_var_37,count_var_38,count_var_39,count_var_40,count_var_41,count_var_42,count_var_43,count_var_44,count_var_45,count_var_46,count_var_47,count_var_48,count_var_49,count_var_50,count_var_51,count_var_52,count_var_53,count_var_54,count_var_55,count_var_56,count_var_57,count_var_58,count_var_59,count_var_60,count_var_61,count_var_62,count_var_63,count_var_64,count_var_65,count_var_66,count_var_67,count_var_68,count_var_69,count_var_70,count_var_71,count_var_72,count_var_73,count_var_74,count_var_75,count_var_76,count_var_77,count_var_78,count_var_79,count_var_80,count_var_81,count_var_82,count_var_83,count_var_84,count_var_85,count_var_86,count_var_87,count_var_88,count_var_89,count_var_90,count_var_91,count_var_92,count_var_93,count_var_94,count_var_95,count_var_96,count_var_97,count_var_98,count_var_99,count_var_100,count_var_101,count_var_102,count_var_103,count_var_104,count_var_105,count_var_106,count_var_107,count_var_108,count_var_109,count_var_110,count_var_111,count_var_112,count_var_113,count_var_114,count_var_115,count_var_116,count_var_117,count_var_118,count_var_119,count_var_120,count_var_121,count_var_122,count_var_123,count_var_124,count_var_125,count_var_126,count_var_127,count_var_128,count_var_129,count_var_130,count_var_131,count_var_132,count_var_133,count_var_134,count_var_135,count_var_136,count_var_137,count_var_138,count_var_139,count_var_140,count_var_141,count_var_142,count_var_143,count_var_144,count_var_145,count_var_146,count_var_147,count_var_148,count_var_149,count_var_150,count_var_151,count_var_152,count_var_153,count_var_154,count_var_155,count_var_156,count_var_157,count_var_158,count_var_159,count_var_160,count_var_161,count_var_162,count_var_163,count_var_164,count_var_165,count_var_166,count_var_167,count_var_168,count_var_169,count_var_170,count_var_171,count_var_172,count_var_173,count_var_174,count_var_175,count_var_176,count_var_177,count_var_178,count_var_179,count_var_180,count_var_181,count_var_182,count_var_183,count_var_184,count_var

## count stats

In [15]:
count_var_stats_cols = [f'count_var_{stat}' for stat in ('mean','std','max','nunique')]

In [16]:
full['count_var_mean'] = full[count_var_cols].mean(axis=1)
full['count_var_std'] = full[count_var_cols].std(axis=1)
full['count_var_max'] = full[count_var_cols].max(axis=1)
full['count_var_nunique'] = full[count_var_cols].nunique(axis=1)

In [17]:
full.head()

,ID_code,target,test,var_0,var_1,var_10,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109,var_11,var_110,var_111,var_112,var_113,var_114,var_115,var_116,var_117,var_118,var_119,var_12,var_120,var_121,var_122,var_123,var_124,var_125,var_126,var_127,var_128,var_129,var_13,var_130,var_131,var_132,var_133,var_134,var_135,var_136,var_137,var_138,var_139,var_14,var_140,var_141,var_142,var_143,var_144,var_145,var_146,var_147,var_148,var_149,var_15,var_150,var_151,var_152,var_153,var_154,var_155,var_156,var_157,var_158,var_159,var_16,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_17,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_18,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_19,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,var_2,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_3,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_4,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_5,var_50,var_51,var_52,var_53,var_54,var_55,var_56,var_57,var_58,var_59,var_6,var_60,var_61,var_62,var_63,var_64,var_65,var_66,var_67,var_68,var_69,var_7,var_70,var_71,var_72,var_73,var_74,var_75,var_76,var_77,var_78,var_79,var_8,var_80,var_81,var_82,var_83,var_84,var_85,var_86,var_87,var_88,var_89,var_9,var_90,var_91,var_92,var_93,var_94,var_95,var_96,var_97,var_98,var_99,count_var_0,count_var_1,count_var_2,count_var_3,count_var_4,count_var_5,count_var_6,count_var_7,count_var_8,count_var_9,count_var_10,count_var_11,count_var_12,count_var_13,count_var_14,count_var_15,count_var_16,count_var_17,count_var_18,count_var_19,count_var_20,count_var_21,count_var_22,count_var_23,count_var_24,count_var_25,count_var_26,count_var_27,count_var_28,count_var_29,count_var_30,count_var_31,count_var_32,count_var_33,count_var_34,count_var_35,count_var_36,count_var_37,count_var_38,count_var_39,count_var_40,count_var_41,count_var_42,count_var_43,count_var_44,count_var_45,count_var_46,count_var_47,count_var_48,count_var_49,count_var_50,count_var_51,count_var_52,count_var_53,count_var_54,count_var_55,count_var_56,count_var_57,count_var_58,count_var_59,count_var_60,count_var_61,count_var_62,count_var_63,count_var_64,count_var_65,count_var_66,count_var_67,count_var_68,count_var_69,count_var_70,count_var_71,count_var_72,count_var_73,count_var_74,count_var_75,count_var_76,count_var_77,count_var_78,count_var_79,count_var_80,count_var_81,count_var_82,count_var_83,count_var_84,count_var_85,count_var_86,count_var_87,count_var_88,count_var_89,count_var_90,count_var_91,count_var_92,count_var_93,count_var_94,count_var_95,count_var_96,count_var_97,count_var_98,count_var_99,count_var_100,count_var_101,count_var_102,count_var_103,count_var_104,count_var_105,count_var_106,count_var_107,count_var_108,count_var_109,count_var_110,count_var_111,count_var_112,count_var_113,count_var_114,count_var_115,count_var_116,count_var_117,count_var_118,count_var_119,count_var_120,count_var_121,count_var_122,count_var_123,count_var_124,count_var_125,count_var_126,count_var_127,count_var_128,count_var_129,count_var_130,count_var_131,count_var_132,count_var_133,count_var_134,count_var_135,count_var_136,count_var_137,count_var_138,count_var_139,count_var_140,count_var_141,count_var_142,count_var_143,count_var_144,count_var_145,count_var_146,count_var_147,count_var_148,count_var_149,count_var_150,count_var_151,count_var_152,count_var_153,count_var_154,count_var_155,count_var_156,count_var_157,count_var_158,count_var_159,count_var_160,count_var_161,count_var_162,count_var_163,count_var_164,count_var_165,count_var_166,count_var_167,count_var_168,count_var_169,count_var_170,count_var_171,count_var_172,count_var_173,count_var_174,count_var_175,count_var_176,count_var_177,count_var_178,count_var_179,count_var_180,count_var_181,count_var_182,count_var_183,count_var_184,count_var

## count on count

In [18]:
count_count_var_cols = [f'count_{col}' for col in count_var_cols]

In [19]:
for col in tqdm(count_var_cols):
    gdf = full.groupby(col)[col].count()
    full[f'count_{col}'] = full[col].map(gdf)

100%|██████████| 200/200 [03:35<00:00,  1.55s/it]


In [20]:
full.head()

,ID_code,target,test,var_0,var_1,var_10,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109,var_11,var_110,var_111,var_112,var_113,var_114,var_115,var_116,var_117,var_118,var_119,var_12,var_120,var_121,var_122,var_123,var_124,var_125,var_126,var_127,var_128,var_129,var_13,var_130,var_131,var_132,var_133,var_134,var_135,var_136,var_137,var_138,var_139,var_14,var_140,var_141,var_142,var_143,var_144,var_145,var_146,var_147,var_148,var_149,var_15,var_150,var_151,var_152,var_153,var_154,var_155,var_156,var_157,var_158,var_159,var_16,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_17,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_18,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_19,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,var_2,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_3,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_4,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_5,var_50,var_51,var_52,var_53,var_54,var_55,var_56,var_57,var_58,var_59,var_6,var_60,var_61,var_62,var_63,var_64,var_65,var_66,var_67,var_68,var_69,var_7,var_70,var_71,var_72,var_73,var_74,var_75,var_76,var_77,var_78,var_79,var_8,var_80,var_81,var_82,var_83,var_84,var_85,var_86,var_87,var_88,var_89,var_9,var_90,var_91,var_92,var_93,var_94,var_95,var_96,var_97,var_98,var_99,count_var_0,count_var_1,count_var_2,count_var_3,count_var_4,count_var_5,count_var_6,count_var_7,count_var_8,count_var_9,count_var_10,count_var_11,count_var_12,count_var_13,count_var_14,count_var_15,count_var_16,count_var_17,count_var_18,count_var_19,count_var_20,count_var_21,count_var_22,count_var_23,count_var_24,count_var_25,count_var_26,count_var_27,count_var_28,count_var_29,count_var_30,count_var_31,count_var_32,count_var_33,count_var_34,count_var_35,count_var_36,count_var_37,count_var_38,count_var_39,count_var_40,count_var_41,count_var_42,count_var_43,count_var_44,count_var_45,count_var_46,...,count_var_154,count_var_155,count_var_156,count_var_157,count_var_158,count_var_159,count_var_160,count_var_161,count_var_162,count_var_163,count_var_164,count_var_165,count_var_166,count_var_167,count_var_168,count_var_169,count_var_170,count_var_171,count_var_172,count_var_173,count_var_174,count_var_175,count_var_176,count_var_177,count_var_178,count_var_179,count_var_180,count_var_181,count_var_182,count_var_183,count_var_184,count_var_185,count_var_186,count_var_187,count_var_188,count_var_189,count_var_190,count_var_191,count_var_192,count_var_193,count_var_194,count_var_195,count_var_196,count_var_197,count_var_198,count_var_199,count_var_mean,count_var_std,count_var_max,count_var_nunique,count_count_var_0,count_count_var_1,count_count_var_2,count_count_var_3,count_count_var_4,count_count_var_5,count_count_var_6,count_count_var_7,count_count_var_8,count_count_var_9,count_count_var_10,count_count_var_11,count_count_var_12,count_count_var_13,count_count_var_14,count_count_var_15,count_count_var_16,count_count_var_17,count_count_var_18,count_count_var_19,count_count_var_20,count_count_var_21,count_count_var_22,count_count_var_23,count_count_var_24,count_count_var_25,count_count_var_26,count_count_var_27,count_count_var_28,count_count_var_29,count_count_var_30,count_count_var_31,count_count_var_32,count_count_var_33,count_count_var_34,count_count_var_35,count_count_var_36,count_count_var_37,count_count_var_38,count_count_var_39,count_count_var_40,count_count_var_41,count_count_var_42,count_count_var_43,count_count_var_44,count_count_var_45,count_count_var_46,count_count_var_47,count_count_var_48,count_count_var_49,count_count_var_50,count_count_var_51,count_count_var_52,count_count_var_53,count_count_var_54,count_count_var_55,count_count_var_56,count_count_var_57,count_count_var_58,count_count_var_59,count_count_var_60,count_count_var_61,count_count

## count count stats

In [21]:
count_count_var_stats_cols = [f'count_count_var_{stat}' for stat in ('mean','std','max','min')]

In [22]:
full['count_count_var_mean'] = full[count_count_var_cols].mean(axis=1)
full['count_count_var_std'] = full[count_count_var_cols].std(axis=1)
full['count_count_var_max'] = full[count_count_var_cols].max(axis=1)
full['count_count_var_min'] = full[count_count_var_cols].min(axis=1)

In [23]:
full.head()

,ID_code,target,test,var_0,var_1,var_10,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109,var_11,var_110,var_111,var_112,var_113,var_114,var_115,var_116,var_117,var_118,var_119,var_12,var_120,var_121,var_122,var_123,var_124,var_125,var_126,var_127,var_128,var_129,var_13,var_130,var_131,var_132,var_133,var_134,var_135,var_136,var_137,var_138,var_139,var_14,var_140,var_141,var_142,var_143,var_144,var_145,var_146,var_147,var_148,var_149,var_15,var_150,var_151,var_152,var_153,var_154,var_155,var_156,var_157,var_158,var_159,var_16,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_17,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_18,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_19,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,var_2,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_3,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_4,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_5,var_50,var_51,var_52,var_53,var_54,var_55,var_56,var_57,var_58,var_59,var_6,var_60,var_61,var_62,var_63,var_64,var_65,var_66,var_67,var_68,var_69,var_7,var_70,var_71,var_72,var_73,var_74,var_75,var_76,var_77,var_78,var_79,var_8,var_80,var_81,var_82,var_83,var_84,var_85,var_86,var_87,var_88,var_89,var_9,var_90,var_91,var_92,var_93,var_94,var_95,var_96,var_97,var_98,var_99,count_var_0,count_var_1,count_var_2,count_var_3,count_var_4,count_var_5,count_var_6,count_var_7,count_var_8,count_var_9,count_var_10,count_var_11,count_var_12,count_var_13,count_var_14,count_var_15,count_var_16,count_var_17,count_var_18,count_var_19,count_var_20,count_var_21,count_var_22,count_var_23,count_var_24,count_var_25,count_var_26,count_var_27,count_var_28,count_var_29,count_var_30,count_var_31,count_var_32,count_var_33,count_var_34,count_var_35,count_var_36,count_var_37,count_var_38,count_var_39,count_var_40,count_var_41,count_var_42,count_var_43,count_var_44,count_var_45,count_var_46,...,count_var_158,count_var_159,count_var_160,count_var_161,count_var_162,count_var_163,count_var_164,count_var_165,count_var_166,count_var_167,count_var_168,count_var_169,count_var_170,count_var_171,count_var_172,count_var_173,count_var_174,count_var_175,count_var_176,count_var_177,count_var_178,count_var_179,count_var_180,count_var_181,count_var_182,count_var_183,count_var_184,count_var_185,count_var_186,count_var_187,count_var_188,count_var_189,count_var_190,count_var_191,count_var_192,count_var_193,count_var_194,count_var_195,count_var_196,count_var_197,count_var_198,count_var_199,count_var_mean,count_var_std,count_var_max,count_var_nunique,count_count_var_0,count_count_var_1,count_count_var_2,count_count_var_3,count_count_var_4,count_count_var_5,count_count_var_6,count_count_var_7,count_count_var_8,count_count_var_9,count_count_var_10,count_count_var_11,count_count_var_12,count_count_var_13,count_count_var_14,count_count_var_15,count_count_var_16,count_count_var_17,count_count_var_18,count_count_var_19,count_count_var_20,count_count_var_21,count_count_var_22,count_count_var_23,count_count_var_24,count_count_var_25,count_count_var_26,count_count_var_27,count_count_var_28,count_count_var_29,count_count_var_30,count_count_var_31,count_count_var_32,count_count_var_33,count_count_var_34,count_count_var_35,count_count_var_36,count_count_var_37,count_count_var_38,count_count_var_39,count_count_var_40,count_count_var_41,count_count_var_42,count_count_var_43,count_count_var_44,count_count_var_45,count_count_var_46,count_count_var_47,count_count_var_48,count_count_var_49,count_count_var_50,count_count_var_51,count_count_var_52,count_count_var_53,count_count_var_54,count_count_var_55,count_count_var_56,count_count_var_57,count_count_var_58,count_count_var_59,count_count_var_60,count_count_var_61,count_count_var_62,count_count_var_63,count_count_var_64,count_coun

# Formatting

In [24]:
tr = full.loc[full['test'] == 0, :]
te = full.loc[full['test'] == 1, :]

In [25]:
te_merge = pd.concat([test_df_sync, te], axis=0)

In [26]:
no_use_cols = [
    'target',
    'test',
#     *var_cols,
#     *count_var_cols,
#     *count_var_stats_cols,
#     *count_count_var_cols,
#     *count_count_var_stats_cols,
]

In [27]:
use_cols = [col for col in tr.columns if col not in no_use_cols]

In [28]:
train_X = tr.loc[:, use_cols + ['target']]
test_X = te_merge.loc[:, use_cols]

In [29]:
train_X.head()

,ID_code,var_0,var_1,var_10,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109,var_11,var_110,var_111,var_112,var_113,var_114,var_115,var_116,var_117,var_118,var_119,var_12,var_120,var_121,var_122,var_123,var_124,var_125,var_126,var_127,var_128,var_129,var_13,var_130,var_131,var_132,var_133,var_134,var_135,var_136,var_137,var_138,var_139,var_14,var_140,var_141,var_142,var_143,var_144,var_145,var_146,var_147,var_148,var_149,var_15,var_150,var_151,var_152,var_153,var_154,var_155,var_156,var_157,var_158,var_159,var_16,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_17,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_18,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_19,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,var_2,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_3,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_4,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_5,var_50,var_51,var_52,var_53,var_54,var_55,var_56,var_57,var_58,var_59,var_6,var_60,var_61,var_62,var_63,var_64,var_65,var_66,var_67,var_68,var_69,var_7,var_70,var_71,var_72,var_73,var_74,var_75,var_76,var_77,var_78,var_79,var_8,var_80,var_81,var_82,var_83,var_84,var_85,var_86,var_87,var_88,var_89,var_9,var_90,var_91,var_92,var_93,var_94,var_95,var_96,var_97,var_98,var_99,count_var_0,count_var_1,count_var_2,count_var_3,count_var_4,count_var_5,count_var_6,count_var_7,count_var_8,count_var_9,count_var_10,count_var_11,count_var_12,count_var_13,count_var_14,count_var_15,count_var_16,count_var_17,count_var_18,count_var_19,count_var_20,count_var_21,count_var_22,count_var_23,count_var_24,count_var_25,count_var_26,count_var_27,count_var_28,count_var_29,count_var_30,count_var_31,count_var_32,count_var_33,count_var_34,count_var_35,count_var_36,count_var_37,count_var_38,count_var_39,count_var_40,count_var_41,count_var_42,count_var_43,count_var_44,count_var_45,count_var_46,count_var_47,count_var_48,...,count_var_159,count_var_160,count_var_161,count_var_162,count_var_163,count_var_164,count_var_165,count_var_166,count_var_167,count_var_168,count_var_169,count_var_170,count_var_171,count_var_172,count_var_173,count_var_174,count_var_175,count_var_176,count_var_177,count_var_178,count_var_179,count_var_180,count_var_181,count_var_182,count_var_183,count_var_184,count_var_185,count_var_186,count_var_187,count_var_188,count_var_189,count_var_190,count_var_191,count_var_192,count_var_193,count_var_194,count_var_195,count_var_196,count_var_197,count_var_198,count_var_199,count_var_mean,count_var_std,count_var_max,count_var_nunique,count_count_var_0,count_count_var_1,count_count_var_2,count_count_var_3,count_count_var_4,count_count_var_5,count_count_var_6,count_count_var_7,count_count_var_8,count_count_var_9,count_count_var_10,count_count_var_11,count_count_var_12,count_count_var_13,count_count_var_14,count_count_var_15,count_count_var_16,count_count_var_17,count_count_var_18,count_count_var_19,count_count_var_20,count_count_var_21,count_count_var_22,count_count_var_23,count_count_var_24,count_count_var_25,count_count_var_26,count_count_var_27,count_count_var_28,count_count_var_29,count_count_var_30,count_count_var_31,count_count_var_32,count_count_var_33,count_count_var_34,count_count_var_35,count_count_var_36,count_count_var_37,count_count_var_38,count_count_var_39,count_count_var_40,count_count_var_41,count_count_var_42,count_count_var_43,count_count_var_44,count_count_var_45,count_count_var_46,count_count_var_47,count_count_var_48,count_count_var_49,count_count_var_50,count_count_var_51,count_count_var_52,count_count_var_53,count_count_var_54,count_count_var_55,count_count_var_56,count_count_var_57,count_count_var_58,count_count_var_59,count_count_var_60,count_count_var_61,count_count_var_62,count_count_var_63,count_count_var_64,count_coun

In [30]:
test_X.head()

,ID_code,var_0,var_1,var_10,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109,var_11,var_110,var_111,var_112,var_113,var_114,var_115,var_116,var_117,var_118,var_119,var_12,var_120,var_121,var_122,var_123,var_124,var_125,var_126,var_127,var_128,var_129,var_13,var_130,var_131,var_132,var_133,var_134,var_135,var_136,var_137,var_138,var_139,var_14,var_140,var_141,var_142,var_143,var_144,var_145,var_146,var_147,var_148,var_149,var_15,var_150,var_151,var_152,var_153,var_154,var_155,var_156,var_157,var_158,var_159,var_16,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_17,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_18,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_19,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,var_2,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_3,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_4,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_5,var_50,var_51,var_52,var_53,var_54,var_55,var_56,var_57,var_58,var_59,var_6,var_60,var_61,var_62,var_63,var_64,var_65,var_66,var_67,var_68,var_69,var_7,var_70,var_71,var_72,var_73,var_74,var_75,var_76,var_77,var_78,var_79,var_8,var_80,var_81,var_82,var_83,var_84,var_85,var_86,var_87,var_88,var_89,var_9,var_90,var_91,var_92,var_93,var_94,var_95,var_96,var_97,var_98,var_99,count_var_0,count_var_1,count_var_2,count_var_3,count_var_4,count_var_5,count_var_6,count_var_7,count_var_8,count_var_9,count_var_10,count_var_11,count_var_12,count_var_13,count_var_14,count_var_15,count_var_16,count_var_17,count_var_18,count_var_19,count_var_20,count_var_21,count_var_22,count_var_23,count_var_24,count_var_25,count_var_26,count_var_27,count_var_28,count_var_29,count_var_30,count_var_31,count_var_32,count_var_33,count_var_34,count_var_35,count_var_36,count_var_37,count_var_38,count_var_39,count_var_40,count_var_41,count_var_42,count_var_43,count_var_44,count_var_45,count_var_46,count_var_47,count_var_48,...,count_var_158,count_var_159,count_var_160,count_var_161,count_var_162,count_var_163,count_var_164,count_var_165,count_var_166,count_var_167,count_var_168,count_var_169,count_var_170,count_var_171,count_var_172,count_var_173,count_var_174,count_var_175,count_var_176,count_var_177,count_var_178,count_var_179,count_var_180,count_var_181,count_var_182,count_var_183,count_var_184,count_var_185,count_var_186,count_var_187,count_var_188,count_var_189,count_var_190,count_var_191,count_var_192,count_var_193,count_var_194,count_var_195,count_var_196,count_var_197,count_var_198,count_var_199,count_var_mean,count_var_std,count_var_max,count_var_nunique,count_count_var_0,count_count_var_1,count_count_var_2,count_count_var_3,count_count_var_4,count_count_var_5,count_count_var_6,count_count_var_7,count_count_var_8,count_count_var_9,count_count_var_10,count_count_var_11,count_count_var_12,count_count_var_13,count_count_var_14,count_count_var_15,count_count_var_16,count_count_var_17,count_count_var_18,count_count_var_19,count_count_var_20,count_count_var_21,count_count_var_22,count_count_var_23,count_count_var_24,count_count_var_25,count_count_var_26,count_count_var_27,count_count_var_28,count_count_var_29,count_count_var_30,count_count_var_31,count_count_var_32,count_count_var_33,count_count_var_34,count_count_var_35,count_count_var_36,count_count_var_37,count_count_var_38,count_count_var_39,count_count_var_40,count_count_var_41,count_count_var_42,count_count_var_43,count_count_var_44,count_count_var_45,count_count_var_46,count_count_var_47,count_count_var_48,count_count_var_49,count_count_var_50,count_count_var_51,count_count_var_52,count_count_var_53,count_count_var_54,count_count_var_55,count_count_var_56,count_count_var_57,count_count_var_58,count_count_var_59,count_count_var_60,count_count_var_61,count_count_var_62,count_count_var_63,count_count_var

In [31]:
train_X.shape

(200000, 610)

In [32]:
test_X.shape

(200000, 609)

In [33]:
dir_path = Path(f'../../results/inputs/count')
dir_path.mkdir(parents=True, exist_ok=True)

In [34]:
%%time
train_X.to_pickle(dir_path.joinpath('train_X.pkl'))
test_X.to_pickle(dir_path.joinpath('test_X.pkl'))

CPU times: user 2.22 s, sys: 7.49 s, total: 9.71 s
Wall time: 13.9 s
